## how to make the qa system follow up question
## error handling - count the mark for all output - many word then the accuracy will low
## need confidence score and accuracy
## 

A is TONG KIT MING and B age is 15. A age is 34.

question is "who is A ? his age is ?"

In [5]:
import tkinter as tk
from transformers import pipeline
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re  # Import the re module for regular expressions

# Initialize the question answering pipeline with the new model
qa_pipeline = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

# Initialize a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

class QA_System:
    def __init__(self):
        self.context = ""
        self.previous_question = ""
        self.previous_answer = ""

    def update_context(self, new_context):
        self.context += new_context

    def answer_question(self, question):
        if self.previous_question and question.lower() == "why":
            return f"Previous Question: {self.previous_question}\nPrevious Answer: {self.previous_answer}", None
        else:
            try:
                sentences = re.split(r'[.!?]', self.context)  # Split the document into sentences
                max_similarity = -1
                best_sentence = ""
                for sentence in sentences:
                    if sentence.strip():
                        # Calculate TF-IDF vectors for each sentence and the question
                        tfidf_matrix = vectorizer.fit_transform([sentence, question])
                        cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)[0][1]
                        if cosine_sim > max_similarity:
                            max_similarity = cosine_sim
                            best_sentence = sentence
                if max_similarity < 0.10:
                    return "This question does not seem to be based on the provided document.", max_similarity

                answer = qa_pipeline(question=question, context=best_sentence)
                return answer['answer'], max_similarity
            except Exception as e:
                return f"Error: {str(e)}", None

def get_time_of_day():
    now = datetime.now()
    current_time = now.time()
    if current_time.hour < 12:
        return "morning"
    elif 12 <= current_time.hour < 18:
        return "afternoon"
    else:
        return "evening"

def respond_to_greeting(greeting):
    time_of_day = get_time_of_day()
    greeting_lower = greeting.lower()

    greetings_responses = {
        "hi": "Hello, how can I assist you today?",
        "hello": "Hello there!",
        "hey": "Hey! What can I help you with?",
        "good morning": f"Good {time_of_day.capitalize()}, welcome! How may I help you?",
        "good afternoon": f"Good {time_of_day.capitalize()}, welcome! How may I assist you?",
        "good evening": f"Good {time_of_day.capitalize()}, welcome! How can I assist you?",
        "yes": "Great to hear!",
        "good": "I'm glad you think so!",
        "why": "That's an interesting question. Could you provide more context?"
    }

    return greetings_responses.get(greeting_lower, None)

def submit_handler(event=None):
    output_widget.config(state=tk.NORMAL)
    output_widget.delete("1.0", tk.END)

    document = document_input.get("1.0", tk.END).strip()
    input_sentence = question_input.get().strip()

    # Split the input sentence into separate questions using '?' and 'and' as delimiters
    questions = re.split(r'[?]|and', input_sentence)

    for question in questions:
        question = question.strip()
        if question:
            response = respond_to_greeting(question)
            if response:
                output_widget.insert(tk.END, f"{response}\n")
                continue

            if document:
                qa_system.update_context(document)

            answer, cosine_sim = qa_system.answer_question(question)
            if cosine_sim is not None:
                output_widget.insert(tk.END, f"Question: {question.strip()}?\nAnswer: {answer.strip()}\nCosine Similarity: {cosine_sim:.2f}\n\n")
            else:
                output_widget.insert(tk.END, f"Question: {question.strip()}?\nAnswer: {answer.strip()}\n\n")

            qa_system.previous_question = question
            qa_system.previous_answer = answer

    output_widget.insert(tk.END, "What's your next question?\n")
    question_input.delete(0, tk.END)
    output_widget.see("end")

def clear_output():
    output_widget.config(state=tk.NORMAL)
    output_widget.delete("1.0", tk.END)
    output_widget.config(state=tk.DISABLED)

def clear_document():
    document_input.delete("1.0", tk.END)

# Initialize Tkinter
root = tk.Tk()
root.title("Question Answering System")

qa_system = QA_System()

# Styling
root.configure(bg="#f0f0f0")
font_style = ("Arial", 12)

# Create UI elements
document_label = tk.Label(root, text="Document:", font=font_style, bg="#f0f0f0")
document_input = tk.Text(root, height=4, width=50, font=font_style)
question_label = tk.Label(root, text="Question:", font=font_style, bg="#f0f0f0")
question_input = tk.Entry(root, width=50, font=font_style)
output_widget = tk.Text(root, height=10, width=50, font=font_style, state=tk.DISABLED)
clear_button = tk.Button(root, text="Clear Output", command=clear_output)
clear_document_button = tk.Button(root, text="Clear Document", command=clear_document)

# Arrange UI elements using grid layout
document_label.grid(row=0, column=0, sticky="w", padx=10, pady=5)
document_input.grid(row=0, column=1, columnspan=2, padx=10, pady=5)
clear_document_button.grid(row=0, column=3, padx=5, pady=5)
question_label.grid(row=1, column=0, sticky="w", padx=10, pady=5)
question_input.grid(row=1, column=1, columnspan=2, padx=10, pady=5)
output_widget.grid(row=2, column=0, columnspan=3, padx=10, pady=5)
clear_button.grid(row=3, column=0, columnspan=3, pady=5)

# Styling for output widget
output_widget.configure(bg="#ffffff", relief=tk.GROOVE, bd=2)

# Bind the Enter key to submit the question
question_input.bind("<Return>", submit_handler)

# Start the Tkinter event loop
root.mainloop()
 

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
